In [ ]:
'Download Brain Tumor MRI Dataset from Kaggle'
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("masoudnickparvar/brain-tumor-mri-dataset")

# print("Path to dataset files:", path)

In [14]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms

from torch.utils.data import DataLoader, Subset
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image

# torch.set_num_threads(os.cpu_count() or 1)
# torch.set_num_interop_threads(max(1, (os.cpu_count() or 1)//2))

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.backends.cudnn.benchmark = True
device

device(type='cpu')

In [ ]:
num_workers = 4 if device.type == 'cuda' else 0
pin = device.type == 'cuda'


# # for rgb images
# tf= transforms.Compose([
#     transforms.Resize((128,128)),
#     transforms.ToTensor(),
#     transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
# ])

# for gray images
tf= transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((64,64)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])


train_load = DataLoader(
    datasets.ImageFolder('Training', tf),
    batch_size=32,
    shuffle=True,
    num_workers= num_workers,
    pin_memory=pin,
    persistent_workers=(num_workers > 0),
    prefetch_factor=4 if num_workers > 0 else None
)


test_load = DataLoader(
    datasets.ImageFolder('Testing', tf),
    batch_size=32,
    shuffle=False,
    num_workers= num_workers,
    pin_memory=pin,
    persistent_workers=(num_workers > 0), # 
    prefetch_factor=4 if num_workers > 0 else None 
)



In [26]:
# Get first image path from your folder
first_class = os.listdir('Training')[0]
first_image_path = os.path.join('Training', first_class, os.listdir(os.path.join('Training', first_class))[0])

# Open with PIL
img = Image.open(first_image_path)
print("Image mode:", img.mode)
print("Image size:", img.size)

Image mode: RGB
Image size: (512, 512)


In [27]:
import cv2 as cv
img = cv.imread(first_image_path)
print("cv2 shape:", img.shape)

cv2 shape: (512, 512, 3)


In [31]:
model= nn.Sequential(
    nn.Conv2d(1,32,3,1,1),  # change to 3 for rgb images
    nn.ReLU(),
    nn.MaxPool2d(3), 

    nn.Conv2d(32,64,3,1,1),
    nn.ReLU(),
    nn.MaxPool2d(3),

    nn.Conv2d(64,128,3,1,1),
    nn.ReLU(),
    nn.MaxPool2d(3),

    nn.AdaptiveAvgPool2d(1), # it is used to reduce the spatial dimensions to 1x1
    nn.Flatten(),

    nn.Linear(128, 256),
    nn.ReLU(),
    nn.Dropout(0.5),

    nn.Linear(256, 4) # 4 classes
).to(device)

In [32]:
optimizer = optim.AdamW(model.parameters(), lr=1e-4)
loss_fn = nn.CrossEntropyLoss()

In [35]:
# Keep 20% of the data for faster experimentation
subset_fraction = 0.5
num_samples = int(len(train_load.dataset) * subset_fraction)
indices = np.random.choice(len(train_load.dataset), num_samples, replace=False)
train_subset  = Subset(train_load.dataset, indices)

train_load_subset = DataLoader(
    train_subset,
    batch_size=train_load.batch_size,
    shuffle=False,                 # use False when providing a subset/sampler
    num_workers=train_load.num_workers,
    pin_memory=train_load.pin_memory,
)


model.train()

epochs = 20
for epoch in range(epochs):

    running_loss = 0

    for x, y in train_load_subset:
        optimizer.zero_grad(set_to_none=True)

        outputs = model(x.to(device))
        loss = loss_fn(outputs, y.to(device))
        loss.backward()

        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss}")


Epoch 1/20, Loss: 76.79315239191055
Epoch 2/20, Loss: 72.20141279697418
Epoch 3/20, Loss: 67.01786115765572
Epoch 4/20, Loss: 63.40051892399788
Epoch 5/20, Loss: 60.435538828372955
Epoch 6/20, Loss: 57.668442249298096
Epoch 7/20, Loss: 55.09694814682007
Epoch 8/20, Loss: 53.20245921611786
Epoch 9/20, Loss: 51.62995371222496
Epoch 10/20, Loss: 49.98560118675232
Epoch 11/20, Loss: 48.11700293421745
Epoch 12/20, Loss: 46.59762218594551
Epoch 13/20, Loss: 46.34744682908058
Epoch 14/20, Loss: 44.84375387430191
Epoch 15/20, Loss: 43.151378989219666
Epoch 16/20, Loss: 43.46009090542793
Epoch 17/20, Loss: 42.07080727815628
Epoch 18/20, Loss: 41.96995264291763
Epoch 19/20, Loss: 41.06304304301739
Epoch 20/20, Loss: 40.97096934914589


In [36]:
model.eval()
test_loss, correct = 0, 0

with torch.no_grad(): # disable gradient calculation because we are in eval mode
    for x, y in test_load:
        outputs = model(x)
        loss = loss_fn(outputs, y)
        test_loss += loss.item()

        preds = outputs.argmax(dim=1)
        correct += (preds == y).sum().item()

test_loss /= len(test_load.dataset)
accuracy = 100 * correct / len(test_load.dataset)

print(f"Test Loss: {test_loss}, Test Accuracy: {accuracy}%")


Test Loss: 0.017122126416701747, Test Accuracy: 77.49809305873379%
